In [ ]:
import os
import time
import csv
import numpy as np
import pandas as pd

import gensim
import sklearn.metrics
import re
import unicodedata
from tqdm import tqdm
import sys
import hashlib
import random

# train_file = 'nlbse23-issue-classification-train.csv'
# test_file = 'nlbse23-issue-classification-test.csv'

# csv.field_size_limit(sys.maxsize) # to avoid error: _csv.Error: field larger than field limit (131072)

# def count_csv_rows(filename):
# 	with open(filename, "r", newline='', encoding='utf-8') as f:
# 		return sum(1 for _ in csv.DictReader(f))

# def print_csv_preview(filename):
# 	print(filename)
# 	print("total rows", count_csv_rows(filename))
# 	display(pd.read_csv(filename, nrows=5))


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# # download the training set if it does not exist
# if not os.path.isfile(train_file):
#   !curl "https://tickettagger.blob.core.windows.net/datasets/{train_file}.tar.gz" | tar -xz

# print_csv_preview(train_file)

# if not os.path.isfile(test_file):
#   !curl "https://tickettagger.blob.core.windows.net/datasets/{test_file}.tar.gz" | tar -xz

# print_csv_preview(test_file)


In [ ]:
# train_df = pd.read_csv(train_file)
# test_df = pd.read_csv(test_file)
#  concatenated_df = pd.concat([train_df,test_df])

In [ ]:
import numpy as np
import pandas as pd

import re
concatenated_df = pd.read_csv("nlbse-2023-issue-report.csv")
concatenated_df.shape
df = concatenated_df[concatenated_df['labels']=='bug']
print(df.shape)
df_unique = df.drop_duplicates(subset='id', keep='first')
print(df_unique.shape)
df = df_unique
df["id"] = df["id"].astype(str)

In [3]:
import math
from collections import Counter

def shannon_entropy(string):
    # Calculate the frequency of each character in the string
    freq = Counter(string)
    
    # Calculate the total number of characters in the string
    total_chars = len(string)
    
    # Calculate the Shannon entropy
    entropy = 0
    for char, count in freq.items():
        probability = count / total_chars
        entropy -= probability * math.log2(probability)
    
    return entropy

In [ ]:
# regex_pattern = r'secret(?!$)'
# p = re.compile(regex_pattern)
# secret_df = df[[try_search(p, x) for x in df['body']]]
# secret_df['body'].to_csv('secret_df.csv')

In [6]:
# !pip3 install openpyxl 
excel_data = pd.read_excel('Secret Regular Expression.xlsx')

# Read the values of the file in the dataframe
regex = pd.DataFrame(excel_data, columns=[
'Pattern_ID','Secret Type',	'Regular Expression','Source'])

In [ ]:

data_dict={}
for j in df.index:
        # if df["id"][j] != "1165939311":
        #         continue
        input_string =    str(df["body"][j])    
        input_string = re.sub(r'[\'"\│]', '', input_string)
        dir_list_clean = re.sub(r'drwx[-\s]*\d+\s+\w+\s+\w+\s+\d+\s+\w+\s+\d+\s+[0-9a-fA-F-]+.*','',input_string)
        shell_code_free_text = re.sub(r'```shell([^`]+)```','',dir_list_clean,flags=re.IGNORECASE)
        shell_code_free_text = re.sub(r'```Shell\s*"([^"]*)"\s*```','',shell_code_free_text,flags=re.IGNORECASE)
        # saved_game_free_text = re.sub(r'```([^`]+)```','',shell_code_free_text) #etay jhamela hobe
        saved_game_free_text = re.sub(r'<details><summary>Saved game</summary>\n\n```(.*?)```', '', shell_code_free_text)
        remove_packages = re.sub(r'(\w+\.)+\w+','',saved_game_free_text)
        java_exp_free_text = re.sub(r'at\s[\w.$]+\.([\w]+)\(([^:]+:\d+)\)','',remove_packages)
        # url_free_text= re.sub(https?://[^\s#]+#[A-Za-z0-9\-]+,'', java_exp_free_text, flags=re.IGNORECASE)
        url_with_fragment_text= re.sub(r'https?://[^\s#]+#[A-Za-z0-9\-\=\+]+','', java_exp_free_text, flags=re.IGNORECASE)
        url_free_text= re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '',url_with_fragment_text)
        commit_free_text= re.sub(r'commit[ ]?(?:id)?[ ]?[:]?[ ]?([0-9a-f]{40})\b', '', url_free_text, flags=re.IGNORECASE)
        file_path_free_text = re.sub(r"/[\w/. :-]+",'',commit_free_text)
        file_path_free_text = re.sub( r'(/[^/\s]+)+','',file_path_free_text)
        sha256_free_text = re.sub(r'sha256\s*[:]?[=]?\s*[a-fA-F0-9]{64}','',file_path_free_text)
        sha1_free_text = re.sub(r'git-tree-sha1\s*=\s*[a-fA-F0-9]+','',sha256_free_text)
        build_id_free_text = re.sub(r'build-id\s*[:]?[=]?\s*([a-fA-F0-9]+)','',sha1_free_text)
        guids_free_text = re.sub(r'GUIDs:\s+([0-9a-fA-F-]+\s+[0-9a-fA-F-]+\s+[0-9a-fA-F-]+)','',build_id_free_text)
        uuids_free_text = re.sub(r'([0-9a-fA-F-]+\s*,\s*[0-9a-fA-F-]+\s*,\s*[0-9a-fA-F-]+)','',guids_free_text)
        event_id_free_text = re.sub(r'<([^>]+)>','',uuids_free_text)
        UUID_free_text = re.sub(r'(?:UUID|GUID|version|id)[\\=:"\'\s]*\b[a-fA-F0-9]{8}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{12}\b'
,'',event_id_free_text,flags=re.IGNORECASE) ##without the prefix so many false positives can be omitted
        hex_free_text = re.sub(r'(?:data|address|id)[\\=:"\'\s]*\b0x[0-9a-fA-F]+\b','',UUID_free_text,flags=re.IGNORECASE) ## deleting hex ids directly can cause issues
        ss_free_text = re.sub(r'Screenshot_(\d{4}[_-]\d{2}[_-]\d{2}[_-]\d{2}[_-]\d{2}[_-]\d{2}[_-]\d{2}[_-]\w+)','',hex_free_text,flags=re.IGNORECASE)
        cleaned_text = ss_free_text
        # file_path = "output.txt"

        # with open(file_path, 'w') as file:
        #                 file.write(cleaned_text)
        data_dict[j] = {'Issue_id':df['id'][j],'Issue_body':cleaned_text}
        # idx = idx+1
    


cleaned_text_data = pd.DataFrame.from_dict(data_dict, "index")
cleaned_text_data

In [ ]:
###recovered
    
idx = 0
data_dict={}
# start = iter*100000
# end = (iter+1)*100000
for i in regex.index:
    print(i,regex['Secret Type'][i]) #, regex['Regular Expression'][i])
    # if i%100==0:
    #     print("checkpoint")
    p = re.compile(regex['Regular Expression'][i])
    
    # print("=====================================================================")
    
    for j in df.index:
        
        cleaned_text = cleaned_text_data.loc[j, 'Issue_body']
            # Now you can use 'cleaned_text' for further processing
       
        matches = re.findall(p,cleaned_text)
        for match in set(matches):
                data_dict[idx] = {'Type': regex['Secret Type'][i], 'Issue_id':df['id'][j],'Candidate_String':match} #,'Entropy':shannon_entropy(match)}
                idx = idx+1
    


data = pd.DataFrame.from_dict(data_dict, "index")
print(data.shape)
data=data.drop_duplicates(subset=["Issue_id", "Candidate_String"], keep='first')
data.to_csv('RQ2-issues-with-secret-key-all-26Sept.csv')


In [ ]:
data.shape

In [ ]:
unique_issue_counts = data['Issue_id'].value_counts().reset_index()
unique_issue_counts.columns = ['Issue_id', 'count']

# Specify the CSV file path where you want to save the data
csv_file_path = 'unique_issue_count-16sept.csv'

# Save the unique issue ID counts to a CSV file
unique_issue_counts.to_csv(csv_file_path, index=False)

print(f'Unique issue ID counts saved to {csv_file_path}')

In [ ]:
## 1376734926 this has commit lets see eta ashe naki ar
#df[df["id"]=="1376693330"].to_csv("demo.csv")
df[df["id"]=="1190623366"]["body"].to_csv("demo.csv")



In [ ]:
import re

text = "Client Version: version.Info{Major:""1"", Minor:""16"", GitVersion:""v1.16.2"", GitCommit:c97fe5036ef3df2967d086711e6c0c405941e14b"

cleaned_text = re.sub(r'commit[ ]?(?:id)?[ ]?[:]?[ ]?([0-9a-f]{40})\b', '', text, flags=re.IGNORECASE)

print(cleaned_text)


In [ ]:
import re

# Sample text containing file paths
test_df = pd.read_csv('demo.csv')

text = str(test_df["body"][0]) 

# Regular expression pattern to match file paths
pattern = r"/[\w/. :-]+"

# Find all matches in the text
print(re.sub(pattern,'', text))

# Print the matches


In [ ]:
import re

text = """
067bff22-df44-4bf4-926b-30f39549e454,
"""

pattern = r'([0-9a-fA-F-]+\s*,\s*[0-9a-fA-F-]+\s*,\s*[0-9a-fA-F-]+)'
print(re.sub(pattern,'',text))

In [ ]:
total_instances = 0
for i in range(1,8):
    temp = pd.read_csv('issues-naive/issues-with-secret-key-'+str(i)+'.csv')
    total_instances = total_instances+ temp.shape[0]
print("RAN ON RAW TEXT: ", total_instances)

In [ ]:
temp = pd.read_csv('issues-refined-without-url-and-git-commit/issues-with-secret-key-6.csv')
temp.shape[0]

In [55]:
import re

text = "Here is a UUID: afefaf06-34e4-4fa8-9878-0c5bbf572d80 and another UUID: 123e4567-e89b-12d3-a456-426614174000"
pattern = r'\b[a-fA-F0-9]{8}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{4}-[a-fA-F0-9]{12}\b'
matches = re.findall(pattern, text)

for match in matches:
    print(match)

In [ ]:
import re

text = "https://prettier.io/playground/#N4Igxg9gdgLgprEAuEA6ARgQzAawOYBOEArlACYC0hmZAlgjABQACYxMAlAATAA6UXQVyy5CJcki4AbWlDiYCVAjXqxG-IZq4wIXArTwALGABoNWwcwDuh2vC4AGAKRmBFrtdv2WbGFwqOqA4AjE4cru4eAA4QBDCYUhQi+ESkZB6+Ee7MMXEJSdgp4unBDs7mghwA3PwAvvwgJiAQUTC00ADOyKAKRFYACgoIXSgJVpgAnl1N6Mq4cDAAypgAtnAAMrJwyABmCR1wM3M4C4tR2LJ4yDAExIcgcCvocGRkL+uYUHjEmHhwAGKxFaYGBtL7IECYdgQRogYwrKQAdS8cA65zAcEWwzstAAbnYJhCwB1piBZAc4v1lHhgbt9vcAFYdAAei0uUjgAEViBB4HSpAcmucCBSIRySbCovpYIjaGQYIZkAAOBxCogHRHKKIQqWouAEXHbJoARx58CpLRGkI6FDkLxesIIcFNtCdVN+tKQewF9wOK1o11uvvZXLN2y99Ka8XQsvliqQACYo8paDIvgBhCArT0PDoAVlhxAOABVMOgRt7BSBcXcAJLkBiLMD6VoAQXIixgEw5-IOtVqQA"

# Regex pattern to detect URLs with fragment identifiers
# pattern = r'https?://\S+#\S+'
pattern = r'https?://[^\s#]+#[A-Za-z0-9\-]+'
matches = re.findall(pattern, text)

for match in matches:
    print(match)

In [ ]:
text = " Module libdouble-conversion.so.3 with build-id:  d162eb67b00f523a3f146b1077ae00fe04d5fb85"
# Define the regex pattern (case-insensitive)
pattern = r'build-id\s*[:]?\s*([a-fA-F0-9]+)'

# Find all matches in the text
matches = re.findall(pattern, text)

# Print the captured build-ids
for match in matches:
    print("Found build-id:", match)

In [ ]:
import re

text = "0x10405c000 - 0x10436bfff +com.utmapp.UTM (2.2.4 - 36) <F55AA24B-A6DD-33E5-880C-8BBE335FAC5D> /Applications/UTM.app/Contents/MacOS/UTM"
pattern = r'([^>]+)'
matches = re.findall(pattern, text)

for match in matches:
    print(match)

In [ ]:
import re

text = "Here is a screenshot: Screenshot_2022-04-20-22-07-32-23_307f4184460b001ebff3ecdd8a4bcf05 and another one Screenshot_2022_11_30_10_15_20_88_1a2b3c4d5e6f7g8h."
matches = re.findall(r'Screenshot_\d{4}_\d{2}_\d{2}_\d{2}_\d{2}_\d{2}_\d{2}_\w{16}', text)

print(matches)

In [ ]:
import re

text = """
GUIDs:        f6b5141d-eeb0-5ff1-b8de-ee4f02b222b7
                       2c4d586a-a2a6-5957-9427-768add598121
                       ebd96940-fa49-5cb1-b1a1-aa2d4dea1416
                       5108e726-44a6-5eca-bd1a-fbbdabbe7bc1
                       31d41b9b-361a-50ad-8bea-8dad3339fa04
"""

pattern = r'GUIDs:\s+([0-9a-fA-F-]+\s+[0-9a-fA-F-]+\s+[0-9a-fA-F-]+)'
matches = re.search(pattern, text)

if matches:
    captured_string = matches.group(0)
    print(captured_string)

things I need to clean further
1. "sha256:a812164ceb48cb62c3217bd6245274e693c624cc2ac0c1b11b4cea96dab054dd"  
2. 3 root root 4096 Oct 16  2020 11af3aea-a3a6-490d-b4e8-871f7bbd1c31                                                
3. git-tree-sha1 = ""e1ba79094cae97b688fb42d31cbbfd63a69706e4""
4. uuid = ""4c555306-a7a7-4459-81d9-ec55ddd5c99a""   
5.        0x10405c000 -        0x10436bfff +com.utmapp.UTM (2.2.4 - 36) <F55AA24B-A6DD-33E5-880C-8BBE335FAC5D> /Applications/UTM.app/Contents/MacOS/UTM
6. build-id 2845e847d99e2c338fc8eef599b018e707c5e811